# Polynomial Regression

Până în acest moment am completat un model de Machine Learning de tipul Linear Regression, am văzut cum se crează un model, pe ce date se antrenează (train set), cum se fac predicții și cum se poate verifica performanța unui model (prin folosirea de metrici, RMSE). Cum se poate îmbunătăți acest model? Un mod de a îmbunătăți un model este prin utilizarea unei relații de grad mai mare între Features. Aici intră în calcul ideea de Polynomial Regression

Polynomial Regression o să ne returneze două idei:

- Non-linear features relationship to label

- Interaction term between features

### Non-linear features realationship to label

Să ne imaginăm că avem un feature din setul de date care nu este liniar (nu există o corelație liniară între puncte, ci mai degrabă una curbată). Având o linie care este mai curbată, este greu să găsim o relație liniară pentru aceste features. În momentul în care se ridică la pătrat acel feature, atunci linia resepctivă o să fie mai apropiată de o linie dreaptă. Dacă numărul cu care se ridică la pătrat este mai mare, atunci linia aceea va deveni tot mai dreatpă. Acest concept ne permită să găsim coeficienții beta de care avem nevoie pentru a crea o regresie liniară. Putem aplica această metodă atunci când corelația dintre features și labels nu este una liniară, prin ridicarea la putere putem să creem mai ușor o linie dreaptă de regresie care să ne găsească un coeficient beta mai corect.

Este de reținut însă faptul că nu fiecare feature o să aibă astfel de relații (relații liniare) la un grad mai mare

### Interaction term

Acest termen face referire la ideea cum că un anumit feature este de interes doar dacă este combinat cu un alt feature. (asta poartă denumirea de synergy în domeniul de buisiness). Să luăm un exemplu din DataFrame-ul pe care l-am avut. Din feature-urile pe care le avem , să zicem că publicitatea din ziare nu are un efect atâta de important pentru vânzările finale, dar combinat cu cel de radio are un efect mai mare. Cum anume se poate verifica dacă două features combinate au o influență mai mare decât individual?

Cel mai simplu mod este de a crea un nou feature care multiplică cele două features de interes (sau care credem că ar avea un efect mai important dacă sunt combinate). Se pot păstra feature-urile inițiale precum și ceea ce a rezultat din multiplicarea acestor features. Din fericire Scikit-Learn ne pune la dispoziție un modul de preprocesare de date unde se găsește și această parte de Polynomial Regression. (poartă denumirea de PolynomialFeatures). Metoda respectiă atât ne creează intracțiunea dintre ele cât și ridicarea la pătrat pentru feicare feature în parte (în funcție de ordinea de degree pe care o alegem). În continuare o să experimentăm aceste terorii în Python

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# reafing the data
df = pd.read_csv('../data/08-Linear-Regression-Models/Advertising.csv')

In [3]:
# printing the head of the data
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


Partea aceasta de Polynomial Features ține mai mult de Feature Regression. Primul pas este să separăm Features de labels din cadrul acestui set de date

In [4]:
# separating the fetures from the labels
X = df.drop(columns='sales')
y = df['sales']

Acum vine partea de Polynomial Regression. Acest procedeu se va face cu ajutorul modului de preprocessing din Scikit-Learn de unde o să importăm PolynomialFeatures

In [5]:
from sklearn.preprocessing import PolynomialFeatures

Acest procedeu funcționează precum un model de Mahine Learning, trebuie să creem o instanță pentru PolynomialFeatures împreună cu anumiți parametrii, dacă dorim să îi schimbăm, după care se antrenează acea instanță utilizând metoda .fit(). Să aruncăm întâi o privire peste parametrii care sunt disponibili

In [6]:
help(PolynomialFeatures)

Help on class PolynomialFeatures in module sklearn.preprocessing._polynomial:

class PolynomialFeatures(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  PolynomialFeatures(degree=2, *, interaction_only=False, include_bias=True, order='C')
 |  
 |  Generate polynomial and interaction features.
 |  
 |  Generate a new feature matrix consisting of all polynomial combinations
 |  of the features with degree less than or equal to the specified degree.
 |  For example, if an input sample is two dimensional and of the form
 |  [a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].
 |  
 |  Read more in the :ref:`User Guide <polynomial_features>`.
 |  
 |  Parameters
 |  ----------
 |  degree : int or tuple (min_degree, max_degree), default=2
 |      If a single int is given, it specifies the maximal degree of the
 |      polynomial features. If a tuple `(min_degree, max_degree)` is passed,
 |      then `min_degree` is the minimum and `max_degree` is the maximum
 |

Ca și parametrii avem următorii: degree, intraction_only, include_bias și order. Degree reprezintă ordinul la care dorim să ridicăm la putere această ecuație, iar în mod default este setat la 2. Pe viitor o să avem o discuție despre cum să alegem cea mai bună valoare pentru acest parametru. Pentru interaction_only putem să specificăm dacă dorim să ne returneze doar interacțiunea dintre featrues, fără a mai ridica la putere fiecare feature în parte.

In [7]:
polynomial_converter = PolynomialFeatures(degree=2, include_bias=False)

Acum, această instanță de PolynomialFeatures așteaptă să fie antrenată pe un anumit set de date. Ce anume înseamnă antrenarea în acest caz, doar preia valorile pentru Features și le analizează, nu o să realizeze acele calcule (transformare) decât doar atunci când îi cerem explicit asta

In [8]:
polynomial_converter.fit(X)

PolynomialFeatures(include_bias=False)

De notat faptul că în acest caz nu trebuie să împărțim setul de date în train și test deorece acesta nu este un model, este doar parte de procesare de features. Prin acest .fit() programul doar a luat datele și s-a uitat peste ele, nu a realizat niciun calcul. Pentru a transforma datele respective în rezultatele unei ecuații polynomial de gradul 2 trebuie să apelăm metoda .transform(). Ca și argument o să îi oferim același set de date ca și la metoda .fit()

In [9]:
polynomial_converter.transform(X)

array([[ 230.1 ,   37.8 ,   69.2 , ..., 1428.84, 2615.76, 4788.64],
       [  44.5 ,   39.3 ,   45.1 , ..., 1544.49, 1772.43, 2034.01],
       [  17.2 ,   45.9 ,   69.3 , ..., 2106.81, 3180.87, 4802.49],
       ...,
       [ 177.  ,    9.3 ,    6.4 , ...,   86.49,   59.52,   40.96],
       [ 283.6 ,   42.  ,   66.2 , ..., 1764.  , 2780.4 , 4382.44],
       [ 232.1 ,    8.6 ,    8.7 , ...,   73.96,   74.82,   75.69]])

Mai sus se poate observa rezultatul acelei metode transform(). Putem să verificăm forma acelui obioect care a fost returnat

In [10]:
polynomial_converter.transform(X).shape

(200, 9)

Putem vedea că ceea ce se retunrează este un obiect care are 200 de rânduri și 9 coloane. Ce anume înseamnă asta, este un rezultat bun? Putem să comparăm cu setul de date inițiat

In [11]:
X.shape

(200, 3)

Ambele seturi de date au același număr de rânduri, ceea ce este bine. Să vedem ce reprezintă acele 9 coloane care au fost returnate. Cum anume s-a ajuns la 9 coloane din 3

In [15]:
results = polynomial_converter.transform(X)

In [16]:
X.iloc[0]

TV           230.1
radio         37.8
newspaper     69.2
Name: 0, dtype: float64

In [17]:
results[0]

array([2.301000e+02, 3.780000e+01, 6.920000e+01, 5.294601e+04,
       8.697780e+03, 1.592292e+04, 1.428840e+03, 2.615760e+03,
       4.788640e+03])

O să ne uităm la valorile din primul element din setul de date inițial și primul element din setul de date care a fost returnat în urma transformării. Au sens valorile respecitve? Primele trei valori din setul de date ce a fost returnat (din primul element) reprezintă valorile pentru fiecare feature din setul de date inițial. Valoarea 2.301000e+02 practiv înseamnă să mutăm virgula cu două valori spre dreapta, ceea ce rezultă în 230.1000, care este la fel cu valoarea din TV de la primul element. Celelalte două valori (3.780000e+01, 6.920000e+01) reprezintă radio și newspaper

Întrebarea este ce reprezintă celelalte 6 valori? 3 dintre acestea reprezintă interacțiunea dintre features, adică valorile a două features înmulțite. Să vedem ce se retunrează dacă se înmulțește (interacțiune) dintre TV și radio

In [18]:
230.1 * 37.8

8697.779999999999

Dacă ne uităm în setul de date returnat, putem vedea că există valoarea 8.697780e+03, de unde acel +03 reprezintă numărul din față înmulțit cu 10 la puterea a 3-a, adică se mută virgula cu 3 valori înspre dreapta, care rezultă în 8697.78 ceea ce reprezintă defapt interacțiunea dintre TV și radio. Trei dintre aceste șase valori rămase reprezintă această interacțiune. Ultimele trei valori care au rămas reprezintă valoarea feature-ului ridicată la puterea 2 (valoarea care s-a ales pentru parametrul degree)

In [19]:
230.1 ** 2

52946.009999999995

5.294601e+04 reprezintă valoare pentru 230.1 ridicată la pătrat. Prin urmare, este corect să avem un număr de 9 coloane care ne sunt returnate după ce datele au fost transformate.

Să revenim la partea de transformare. Este comun ca un anumit modul din Scikit-Learn să îți ceară prima dată datele ca să le vizualizeze (metoda .fit()) iar abia după să îi specifici să facă transformarea pentru datele respective, aceste module ne pun la dispoziție metoda .fit_transform() care face aceste lucruri în același timp, deoarece este atât de des întâlnit să se folosească fit împreună cu transform

In [20]:
poly_results = polynomial_converter.fit_transform(X)

In [21]:
poly_results

array([[ 230.1 ,   37.8 ,   69.2 , ..., 1428.84, 2615.76, 4788.64],
       [  44.5 ,   39.3 ,   45.1 , ..., 1544.49, 1772.43, 2034.01],
       [  17.2 ,   45.9 ,   69.3 , ..., 2106.81, 3180.87, 4802.49],
       ...,
       [ 177.  ,    9.3 ,    6.4 , ...,   86.49,   59.52,   40.96],
       [ 283.6 ,   42.  ,   66.2 , ..., 1764.  , 2780.4 , 4382.44],
       [ 232.1 ,    8.6 ,    8.7 , ...,   73.96,   74.82,   75.69]])

Această transformare se va face pe întreg setul de date deoarece constă în partea de pregătire a datelor. După ce se face această pregătire, transformare atunci se va împărți setul de date în train-test pentru a fi antrenat un model, a se face predicții cu acel model și a se evalua modelul.

## Recapitulare

În cadrul acestei părți am învățat următoarele lucruri:

    1. Posibilitatea de a utiliza un Polynomial Regression pentru situațiile în care nu se potrivește un Linear Regression. Atunci când avem o linie care nu este dreaptă, ci curbată

    2. Ce anume se retunrează în urma unui Polynomial Regression

        interacțiunea dintre features

        valorea ridicată la pătrat pentru fiecare feature

    3. Cum să importăm această regresie

        from sklearn.preprocessing import PolynomialFeatures

    4. Cum să instanțiem o astfel de regresie

        polynomial_conerter = PolynomialFeatures(degree=2, include_bias=False)

    5. Cum să specificăm pe ce date o să se realizeze această transformare

        polynomial_converter.fit(X)

            Această metodă de fit doar se utiă peste datele pe care o să le utilizăm

    6. Cum să transformăm datele

        polynomial_converter.transform(X)

    7. Posibilitatea de a folosi .fit() și .transform() cu o singură metodă

        polynomial_converter.fit_transform(X)